In [0]:
from pyspark.sql import *
from pyspark import SparkConf

In [0]:
dbutils.fs.ls('/FileStore/tables/')

Out[49]: [FileInfo(path='dbfs:/FileStore/tables/1_registered_users_1.csv', name='1_registered_users_1.csv', size=266, modificationTime=1706195933000),
 FileInfo(path='dbfs:/FileStore/tables/1_registered_users_2.csv', name='1_registered_users_2.csv', size=263, modificationTime=1706195933000),
 FileInfo(path='dbfs:/FileStore/tables/2_user_info_1.json', name='2_user_info_1.json', size=2913, modificationTime=1706195933000),
 FileInfo(path='dbfs:/FileStore/tables/2_user_info_2.json', name='2_user_info_2.json', size=2482, modificationTime=1706195933000),
 FileInfo(path='dbfs:/FileStore/tables/3_bpm_1.json', name='3_bpm_1.json', size=39990667, modificationTime=1706195982000),
 FileInfo(path='dbfs:/FileStore/tables/3_bpm_2.json', name='3_bpm_2.json', size=39994963, modificationTime=1706195982000),
 FileInfo(path='dbfs:/FileStore/tables/4_workout_1.json', name='4_workout_1.json', size=3319, modificationTime=1706195983000),
 FileInfo(path='dbfs:/FileStore/tables/4_workout_2.json', name='4_workou

In [0]:
conf = SparkConf()
conf.set("spark.app.name","targaryen")
conf.set("spark.master","local[*]")
    
spark = SparkSession.builder.config(conf=conf).getOrCreate()  
    
# schema = "user_id long , device_id long , mac_address string , registration_timestamp long"
df = (spark.read
        .format("csv") 
        .option("header","true") 
        .option("inferschema","true") 
        .load("/FileStore/tables/5_gym_logins_*")
)

In [0]:
df.columns

Out[51]: ['mac_address', 'gym', 'login', 'logout']

In [0]:
display(df)

mac_address,gym,login,logout
4c:c5:9f:cb:13:bd,5,1678521600,1678526100
ae:ec:f6:48:ca:f7,1,1678522500,1678525200
36:1f:d9:d3:e8:0d,3,1678522500,1678527000
14:cd:d6:db:70:f6,5,1678523400,1678527600
57:24:ac:8c:75:ea,1,1678524000,1678528500
36:1f:d9:d3:e8:0d,3,1678561200,1678564800
14:cd:d6:db:70:f6,5,1678562400,1678565700
57:24:ac:8c:75:ea,5,1678562880,1678567200
1d:69:69:75:d0:aa,1,1678608000,1678611000
df:f9:dc:5e:e2:a8,1,1678608000,1678611600


In [0]:
df1 = df.select(df.mac_address.cast('string'),df.gym.cast('int'),df.login.cast('timestamp'),df.logout.cast('timestamp'))

In [0]:
from pyspark.sql.functions import window,sum,count
df2 = df1.groupBy(window(df1.login,"30 minutes")).agg(count("*"),sum(df1.gym))
display(df2)

window,count(1),sum(gym)
"List(2023-03-11T08:00:00.000+0000, 2023-03-11T08:30:00.000+0000)",3,9
"List(2023-03-11T19:00:00.000+0000, 2023-03-11T19:30:00.000+0000)",3,13
"List(2023-03-11T08:30:00.000+0000, 2023-03-11T09:00:00.000+0000)",2,6
"List(2023-03-12T08:30:00.000+0000, 2023-03-12T09:00:00.000+0000)",1,5
"List(2023-03-12T08:00:00.000+0000, 2023-03-12T08:30:00.000+0000)",4,10
"List(2023-03-12T18:00:00.000+0000, 2023-03-12T18:30:00.000+0000)",3,7


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("UserLoginLogout").getOrCreate()

# Sample data
data = [
    (1, 'login', '2024-01-25T10:00:00'),
    (1, 'logout', '2024-01-25T12:00:00'),
    (2, 'login', '2024-01-25T11:00:00'),
    (2, 'logout', '2024-01-25T14:00:00'),
    # Add more data as needed
]

# Define the schema for the DataFrame
schema = ["user_id", "type", "time"]

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)

# Pivot the DataFrame to get login and logout times as separate columns
pivoted_df = df.groupBy("user_id").pivot("type").agg({"time": "max"})

# Rename columns for clarity
result_df = pivoted_df.select(
    col("user_id"),
    col("login").alias("logintime"),
    col("logout").alias("logouttime")
)

# Show the result
result_df.show(truncate=False)

# Stop the Spark session


+-------+-------------------+-------------------+
|user_id|logintime          |logouttime         |
+-------+-------------------+-------------------+
|1      |2024-01-25T10:00:00|2024-01-25T12:00:00|
|2      |2024-01-25T11:00:00|2024-01-25T14:00:00|
+-------+-------------------+-------------------+



In [0]:
from pyspark.sql.functions import expr
df1 = (df.select("*").withColumn('logouttime',expr("case when type == 'logout' then time else 0 end")).
      withColumn('logintime',expr("case when type == 'login' then time else 0 end"))
)

In [0]:
display(df1)

user_id,type,time,logouttime,logintime
1,login,2024-01-25T10:00:00,0,2024-01-25T10:00:00
1,logout,2024-01-25T12:00:00,2024-01-25T12:00:00,0
2,login,2024-01-25T11:00:00,0,2024-01-25T11:00:00
2,logout,2024-01-25T14:00:00,2024-01-25T14:00:00,0


In [0]:
from pyspark.sql.functions import expr,when
df1 = (df.select("*").withColumn('logouttime',expr("case when type == 'logout' then time else 0 end")).
      withColumn('logintime',when(df.type == 'login',df.time).otherwise(0))
)

In [0]:
df1.show()

+-------+------+-------------------+-------------------+-------------------+
|user_id|  type|               time|         logouttime|          logintime|
+-------+------+-------------------+-------------------+-------------------+
|      1| login|2024-01-25T10:00:00|                  0|2024-01-25T10:00:00|
|      1|logout|2024-01-25T12:00:00|2024-01-25T12:00:00|                  0|
|      2| login|2024-01-25T11:00:00|                  0|2024-01-25T11:00:00|
|      2|logout|2024-01-25T14:00:00|2024-01-25T14:00:00|                  0|
+-------+------+-------------------+-------------------+-------------------+



In [0]:
from pyspark.sql.functions import expr,when,col,max,count

df2=df1.groupBy("user_id").agg(max("*"))
display(df2)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-1180535565560640>", line 3, in <module>
    df2=df1.groupBy("user_id").agg(max("*"))
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 48, in wrapper
    res = func(*args, **kwargs)
  File "/databricks/spark/python/pyspark/sql/group.py", line 177, in agg
    jdf = self._jgd.agg(exprs[0]._jc, _to_seq(self.session._sc, [c._jc for c in exprs[1:]]))
  File "/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/databricks/spark/python/pyspark/errors/exceptions.py", line 234, in deco
    raise converted from None
pyspark.errors.exceptions.AnalysisException: Invalid usage of '*' in expression 'max'.

During handling of the above exception, another exception occurred:


In [0]:
from pyspark.sql.functions import expr,when,col,max,count
from pyspark.sql import window as win
print(df1.printSchema())

df2 = df1.withColumn('hii' , df1.logouttime.cast('timestamp') + df1.logintime.cast('timestamp'))
# df2=df1.groupBy("user_id").agg(sum(col('logouttime'),col('logintime')))
display(df2)

In [0]:
from pyspark.sql.functions import expr,when
df1 = (df.select("*").
    # withColumn('logouttime',expr("case when type == 'logout' then time else 0 end")).
    withColumn('logouttime',when(df.type == 'logout',df.time).otherwise(None)).
    withColumn('logintime',when(df.type == 'login',df.time).otherwise(None))
)

In [0]:
from pyspark.sql.functions import expr,when,col,max,count,first,last
from pyspark.sql.window import Window as win
print(df1.printSchema())

wn = win.partitionBy('user_id')

df2 = (df1.withColumn('lit',first('logintime').over(wn.orderBy(col('time').asc())))
    .withColumn('lot',first('logouttime').over(wn.orderBy(col('time').desc())))
    .select('user_id','lit','lot')
    .dropDuplicates()
    )


# df2=df1.groupBy("user_id").agg(sum(col('logouttime'),col('logintime')))
display(df2)

root
 |-- user_id: long (nullable = true)
 |-- type: string (nullable = true)
 |-- time: string (nullable = true)
 |-- logouttime: string (nullable = true)
 |-- logintime: string (nullable = true)

None


user_id,lit,lot
1,2024-01-25T10:00:00,2024-01-25T12:00:00
2,2024-01-25T11:00:00,2024-01-25T14:00:00


In [0]:
display(df1)

user_id,type,time,logouttime,logintime
1,login,2024-01-25T10:00:00,null,2024-01-25T10:00:00
1,logout,2024-01-25T12:00:00,2024-01-25T12:00:00,null
2,login,2024-01-25T11:00:00,null,2024-01-25T11:00:00
2,logout,2024-01-25T14:00:00,2024-01-25T14:00:00,null
